<div class="alert alert-success" role="alert">
This work consists of two parts:     
    <ul>
        <li> PART 1 - TRAIN MODEL WITH META DATA!!</li>
        <li> PART 2 - SOME IMAGE RELATED ENHANCEMENT</a></li>
    </ul>
    
</div>

<div class="alert alert-warning">
<strong>Feel free to use it and enjoy!
    I really appreciate if you upvote this notebook. Thank you! </strong>
</div>


In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from fastai.vision.all import *

In [ ]:
set_seed(999, reproducible=True)
BATCH_SIZE = 8
Image_size = 224
epoch = 5
Model_name = "swin_large_patch4_window7_224"
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

In [ ]:
import sys
import fastai
import timm
from timm import create_model
from fastai.vision.all import *
# timm.list_models()
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scale = StandardScaler()
train_df = pd.read_csv(dataset_path/'train.csv')

train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

meta =True
if meta:
    def size_and_shape(row):
        img = Image.open(row['path'])
        return pd.Series([img.size[0], img.size[1], img.size[1]/img.size[0], img.size[0]/img.size[1], os.path.getsize(row['path'])])

    scale = MinMaxScaler()

    train_df[['width', 'height', 'hw_ratio', 'wh_ratio', 'size']] = pd.DataFrame(scale.fit_transform(train_df.apply(size_and_shape, axis=1).values))


In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")

In [ ]:
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")
print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")

In [ ]:
train_df['norm_score'] = train_df['Pawpularity']/100
train_df['norm_score']

In [ ]:
im = Image.open(train_df['path'][1])
width, height = im.size
print(width,height)

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'


In [ ]:
seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
#Sturges' rule
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
# num_bins

In [ ]:
train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
train_df['bins'].hist()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

train_df['fold'] = -1


N_FOLDS = 5
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

train_df.fold.value_counts().plot.bar()

In [ ]:
train_df[train_df['fold']==0].head()
train_df[train_df['fold']==0]['bins'].value_counts()
train_df[train_df['fold']==1]['bins'].value_counts()

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
def get_data(fold):
#     train_df_no_val = train_df.query(f'fold != {fold}')
#     train_df_val = train_df.query(f'fold == {fold}')
    
#     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
#                                valid_pct=0.2, #80-20 train-validation random split
                               valid_col='is_valid', #
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(Image_size, resamples=(Image.BICUBIC, Image.NEAREST)), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))    #pass in batch_tfms
    
    return dls


meta_target = train_df.drop(['norm_score','Pawpularity','path', 'fold', 'bins'],1).keys().tolist()
print("meta len:",len(meta_target))

if meta:
    def get_data(fold):
        train_df_f = train_df.copy()
        # add is_valid for validation fold
        if fold != -1:
            train_df_f['is_valid'] = (train_df_f['fold'] == fold)
            print("Using kfold: ", fold)
        elif fold == -1:
            train_df_f['is_valid'] = (train_df_f['fold'].values < 10 ) 
            print("Using Whole dataset for LR finder")

        paw_block = DataBlock(blocks = (ImageBlock, RegressionBlock, RegressionBlock) ,  # input/ output
                              get_x = [ColReader('path'), ColReader(meta_target)], 
                              get_y = ColReader('norm_score') , 
                              item_tfms = Resize(Image_size, resamples=(Image.BICUBIC, Image.NEAREST)), 
                              batch_tfms = setup_aug_tfms([#Flip(p=0.5),
                                              Brightness(max_lighting=0.2, p=0.75), 
                                              Contrast(max_lighting=0.2, p=0.75), 
                                              Hue(max_hue=0.1, p=0.75), 
                                              Saturation(max_lighting=0.2, p=0.75)])
                              # RandomErasing(p=0.5, sl=0.0, sh=0.3, min_aspect=0.3, max_count=1)


                             )

        paw_dls = paw_block.dataloaders(train_df_f, batch_size=BATCH_SIZE, y_block=RegressionBlock, num_workers=8)
        return paw_dls
    

In [ ]:
#Valid Kfolder size
the_data = get_data(0)
print('finished')
assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

In [ ]:
class PawpularModel(nn.Module):
    def __init__(self, model_name='swin_large_patch4_window7_224', num_classes=[17,1],
                 n_meta_dim=[256, 128], pretrained=True):
        super().__init__()
        out_dim=num_classes[1]
        n_meta_features=num_classes[0]
        self.n_meta_features = n_meta_features
        self.enet = create_model(model_name, pretrained=True, in_chans=3)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        in_ch = self.enet.head.in_features
        if n_meta_features > 0:
            self.meta = nn.Sequential(
                nn.Linear(n_meta_features, n_meta_dim[0]),
                nn.BatchNorm1d(n_meta_dim[0]),
                nn.SiLU(),
                nn.Dropout(p=0.3),
                nn.Linear(n_meta_dim[0], n_meta_dim[1]),
                nn.BatchNorm1d(n_meta_dim[1]),
                nn.SiLU(),
            )
            in_ch += n_meta_dim[1]
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.head = nn.Identity()

    def extract(self, x):
        x = self.enet(x)
        return x

    def forward(self, image, features):
        x = self.extract(image).squeeze(-1).squeeze(-1)
        if self.n_meta_features > 0:
            x_meta = self.meta(features)
            x = torch.cat((x, x_meta), dim=1)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                out = self.myfc(dropout(x))
            else:
                out += self.myfc(dropout(x))
        out /= len(self.dropouts)
        return out

def get_learner(fold_num):
    data = get_data(fold_num)
    if meta:
        print("data.c:", data.c)
        model = PawpularModel(Model_name, num_classes=data.c)
    else:    
        model = create_model(Model_name, pretrained=True, num_classes=data.c)

    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    
    return learn

In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])

if meta:
    def size_and_shape(row):
        img = Image.open(row['path'])
        return pd.Series([img.size[0], img.size[1], img.size[1]/img.size[0], img.size[0]/img.size[1], os.path.getsize(row['path'])])

    scale_test = MinMaxScaler()

    test_df[['width', 'height', 'hw_ratio', 'wh_ratio', 'size']] = pd.DataFrame(scale_test.fit_transform(test_df.apply(size_and_shape, axis=1).values))


train_df['norm_score'] = train_df['Pawpularity']/100

# get_learner(fold_num=-1).lr_find(end_lr=3e-2)
  # SuggestedLRs(valley=0.00017041253158822656
#get_learner(fold_num=0).lr_find(end_lr=3e-2) 

In [ ]:
!ls ../input/baseline-224-mata-v2-1/models

In [ ]:
Path('./models').mkdir(exist_ok=True, parents=True)
!cp ../input/baseline-224-mata-v2-1/models/learn_saved_model_fold_0.pth ./models/
print(0)
!cp ../input/baseline-224-mata-v2-1/models/learn_saved_model_fold_1.pth ./models/
!cp ../input/baseline-224-mata-v2-1/models/learn_saved_model_fold_2.pth ./models/
!cp ../input/baseline-224-mata-v2-1/models/learn_saved_model_fold_3.pth ./models/
!cp ../input/baseline-224-mata-v2-1/models/learn_saved_model_fold_4.pth ./models/

In [ ]:
# import gc
# torch.cuda.empty_cache()
# all_preds = []

# for i in range(N_FOLDS):  # 

#     print(f'Fold {i} results')
    
#     learn = get_learner(fold_num=i)
#     # 2e-5
#     learn.load('learn_saved_model_fold_{}'.format(i))

#     #learn = learn.to_fp32()
    
#     #learn.export(f'model_fold_{i}.pkl')
#     #learn.save(f'model_fold_{i}.pkl')
#     if meta:
#         meta_target = train_df.drop(['norm_score','Pawpularity','path', 'fold', 'bins'], 1).keys().tolist()
#         dls = get_data(i)
#     else:
#         dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
#                                    valid_pct=0.2, #80-20 train-validation random split
#                                    seed=999, #seed
#                                    fn_col='path', #filename/path is in the second column of the DataFrame
#                                    label_col='norm_score', #label is in the first column of the DataFrame
#                                    y_block=RegressionBlock, #The type of target
#                                    bs=BATCH_SIZE, #pass in batch size
#                                    num_workers=8,
#                                    item_tfms=Resize(Image_size, resamples=(Image.BICUBIC, Image.NEAREST)), #pass in item_tfms
#                                    batch_tfms=setup_aug_tfms([Brightness(max_lighting=0.2, p=0.75), 
#                                                         Contrast(max_lighting=0.2, p=0.75), 
#                                                         Hue(max_hue=0.1, p=0.75), 
#                                                         Saturation(max_lighting=0.2, p=0.75), FlipItem(p=.5)
#                                                         ]))   
#          # RandomErasing(p=0.5, sl=0.0, sh=0.3, min_aspect=0.3, max_count=1) 

    
#     test_dl = dls.test_dl(test_df)
    
#     preds, _ = learn.tta(dl=test_dl, n=5, beta=0)    # 'Series' object has no attribute 'widt
#     all_preds.append(preds)
    
#     del learn
#     torch.cuda.empty_cache()
#     gc.collect()

In [ ]:
# np.mean(np.stack(all_preds*100))

In [ ]:
# sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
# preds = np.mean(np.stack(all_preds), axis=0)
# sample_df['Pawpularity'] = preds*100
# sample_df.to_csv('submission.csv',index=False)

In [ ]:
# pd.read_csv('submission.csv')

# SVR head

In [ ]:
using_svr = True
import sys
sys.path.append("../input/tez-lib/")
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from timm import create_model
from fastai.vision.all import *
meta = True

# train part
import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math
import gc
class args:
    batch_size = 16
    image_size = 224
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
set_seed(999, reproducible=True)

dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()
# train_df = pd.read_csv(dataset_path/'train.csv')

from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler#, StandardSaclertrain_df.head()

In [ ]:

class SVR_Model(nn.Module):
    def __init__(self, model_name='swin_large_patch4_window7_224', num_classes=[19,1]):
        super().__init__()
        self.model = create_model(model_name, pretrained=True, in_chans=3)
        #timm.create_model(model_name, pretrained=False, in_chans=3)
        self.model.head = nn.Sequential()#nn.Linear()  # 128 + 17  # self.model.head.in_features, 128+128
        self.dropout = nn.Dropout(0.1)
        self.dense_fc = nn.Linear(num_classes[0], 128)
        self.dense1 = nn.Linear(1536 + 128, 256)
        self.dense2 = nn.Linear(256, num_classes[1])



    def forward(self, image, features, targets=None):
        x1 = self.model(image)
        x = self.dropout(x1)
#         features = nn.sigmoid(self.dense_f1(features))
        x = torch.cat([x, nn.ReLU()(self.dense_fc(features))], dim=1)  
        x = self.dense1(x)
        x = nn.ReLU()(x)       #SiLU
        x = self.dense2(x)
        
        x = torch.cat([x, x1, features], dim=1)
        return x
    

class SVRModel(nn.Module):      
    def __init__(self, model_name='swin_large_patch4_window7_224', num_classes=[17,1],
                 n_meta_dim=[256, 128], pretrained=True):
        super().__init__()
        out_dim=num_classes[1]
        n_meta_features=num_classes[0]
        self.n_meta_features = n_meta_features
        self.enet = create_model(model_name, pretrained=True, in_chans=3)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        in_ch = self.enet.head.in_features
        if n_meta_features > 0:
            self.meta = nn.Sequential(
                nn.Linear(n_meta_features, n_meta_dim[0]),
                nn.BatchNorm1d(n_meta_dim[0]),
                nn.SiLU(),
                nn.Dropout(p=0.3),
                nn.Linear(n_meta_dim[0], n_meta_dim[1]),
                nn.BatchNorm1d(n_meta_dim[1]),
                nn.SiLU(),
            )
            in_ch += n_meta_dim[1]
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.head = nn.Identity()

    def extract(self, x):
        x = self.enet(x)
        return x

    def forward(self, image, features):
        x1 = self.extract(image).squeeze(-1).squeeze(-1)
        if self.n_meta_features > 0:
            x_meta = self.meta(features)
            x = torch.cat((x1, x_meta), dim=1)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                out = self.myfc(dropout(x))
            else:
                out += self.myfc(dropout(x))
        out /= len(self.dropouts)
        
        x = torch.cat([out, x1, features, x_meta], dim=1)
#         print(x.shape)
        return x
    
    
def get_svr_learner(fold_num):
    data = get_data(fold_num)

    print("data.c:", data.c)
    model = SVRModel(Model_name, num_classes=data.c)

    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    
    return learn

In [ ]:
import cuml, pickle
from cuml.svm import SVR
from sklearn.svm import SVR as skSVR
print('RAPIDS version',cuml.__version__,'\n')

In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])

if meta:
    def size_and_shape(row):
        img = Image.open(row['path'])
        return pd.Series([img.size[0], img.size[1], img.size[1]/img.size[0], img.size[0]/img.size[1], os.path.getsize(row['path'])])

#     scale_test = MinMaxScaler()

    test_df[['width', 'height', 'hw_ratio', 'wh_ratio', 'size']] = pd.DataFrame(scale.transform(test_df.apply(size_and_shape, axis=1).values))

train_df['norm_score'] = train_df['Pawpularity']/100


# Train Time

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_skSVR = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_skSVR = []
super_final_oof_true = []
LOAD_SVR_FROM_PATH = None

train_path = pd.read_csv(dataset_path/'train.csv')
tta_n = 5
N_FOLDS = 5

dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'width', 
        'height' ,'hw_ratio','wh_ratio']


for fold_ in range(N_FOLDS):

    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
    dls = get_data(fold_)
    learn = get_svr_learner(fold_num = fold_)

    learn.load('learn_saved_model_fold_{}'.format(fold_))

    
    train_df_f = train_df.copy()
    
    name = f"SVR_fold_{fold_}.pkl" 
    
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        print('Extracting train embedding...')
        train_dl = dls.test_dl(  train_df[train_df['fold']!=fold_] )
        train_predictions, _ = learn.tta(dl=train_dl, n=tta_n, beta=0)   
        
        
        embed = np.array([]).reshape((0,1682))   # 128+ 12
        for preds in train_predictions:
            embed = np.concatenate([embed,preds.unsqueeze(0)],axis=0)
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        
        # C = 1.0
        clf = SVR(C=5.0)  # 20 ? smaller
        skclf = skSVR(C=1.0)    # C=1.0
        
        
        # Expected 7930 rows but got 7929 rows
        clf.fit(embed.astype('float32'), train_df[train_df['fold']!=fold_].norm_score.values.astype('float32'))
        skclf.fit(embed.astype('float32'), train_df[train_df['fold']!=fold_].norm_score.values.astype('float32'))
        
        
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    print('Predicting test...')
 
    test_dl = dls.test_dl( test_df )        
    test_predictions, _ = learn.tta(dl=test_dl, n=tta_n, beta=0)    # 'Series' object has no attribute 'width
        
    final_test_predictions = []
    embed = np.array([]).reshape((0,1682))
    for preds in test_predictions: #tqd
        final_test_predictions.extend(preds[0].ravel().tolist())
        embed = np.concatenate([embed,preds.unsqueeze(0)],axis=0)

    final_test_predictions = [x * 100 for x in final_test_predictions]   # sigmoid(x)
    final_test_predictions2 = clf.predict(embed.astype('float32'))
    final_test_prediction_sklearn = skclf.predict(embed.astype('float32'))
    
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)
    super_final_skSVR.append(final_test_prediction_sklearn)
    ##################
    
    ##################
    # OOF PREDICTIONS
    print('Predicting oof...')
    val_dl = dls.test_dl( train_df[train_df['fold']==fold_] )     
    # sigmoid(): argument 'input' (position 1) must be Tensor, not tuple
    valid_predictions , _ = learn.tta(dl=val_dl, n=tta_n, beta=0)    # 'Series' object has no attribute 'width
        
#     valid_predictions = model.predict(valid_dataset, batch_size=2*args.batch_size, n_jobs=-1)

    final_oof_predictions = []
    embed = np.array([]).reshape((0,1682))
    for preds in valid_predictions:
        final_oof_predictions.extend(preds[0].ravel().tolist())
        embed = np.concatenate([embed,preds.unsqueeze(0)],axis=0)

    final_oof_predictions = [x * 100 for x in final_oof_predictions]    # sigmoid(x)
    final_oof_predictions2 = clf.predict(embed.astype('float32'))  
    final_oof_predictions_sklearn = skclf.predict(embed.astype('float32'))
    super_final_oof_predictions.append(final_oof_predictions)
    super_final_oof_predictions2.append(final_oof_predictions2)
    super_final_oof_skSVR.append(final_oof_predictions_sklearn)
    
    final_oof_true = train_df[train_df['fold']==fold_]['Pawpularity'].values
    super_final_oof_true.append(final_oof_true)
    ##################
    
    ##################
    # COMPUTE RSME
    rsme = np.sqrt( np.mean( (super_final_oof_true[-1] - np.array(super_final_oof_predictions[-1]))**2.0 ) )
    print('NN RSME =',rsme,'\n')
    rsme = np.sqrt( np.mean( (super_final_oof_true[-1] - np.array(super_final_oof_predictions2[-1])*100)**2.0 ) )
    print('SVR RSME =',rsme,'\n')
    rsme = np.sqrt( np.mean( (super_final_oof_true[-1] - np.array(super_final_oof_skSVR[-1])*100)**2.0 ) )
    print('SK SVR RSME =',rsme,'\n')
    
    w = 0.3

    oof2 = (1-w)*np.array(super_final_oof_predictions[-1]) + w*np.array(super_final_oof_predictions2[-1])*100
    rsme = np.sqrt( np.mean( (super_final_oof_true[-1] - oof2)**2.0 ) )
    print('Ensemble RSME =',rsme,'\n')
    

    oof3 = (1-w)*np.array(super_final_oof_predictions[-1]) + 0.5*w*np.array(super_final_oof_predictions2[-1])*100 + \
    0.5*w*np.array(super_final_oof_skSVR[-1])*100
    rsme = np.sqrt( np.mean( (super_final_oof_true[-1] - oof3)**2.0 ) )
    print('With Sk Ensemble RSME =',rsme,'\n')

    del learn, clf

    torch.cuda.empty_cache()

    gc.collect()

In [ ]:
train_df[train_df['fold']==fold_][dense_features] 

Inverse Sigmoid: x = ln(y/(1-y))

In [ ]:
super_final_oof_predictions = np.array(super_final_oof_predictions)
super_final_oof_true = np.array(super_final_oof_true)

print(super_final_oof_predictions.shape)
print(super_final_oof_true.shape)

rsme = np.sqrt( np.mean( (super_final_oof_predictions[-1] - super_final_oof_true[-1])**2.0 ) )
print('RSME =',rsme,'\n')


In [ ]:
true = np.hstack(super_final_oof_true)

oof = np.hstack(super_final_oof_predictions)
rsme = np.sqrt( np.mean( (oof - true)**2.0 ))
print('Overall CV NN head RSME =',rsme)

oof2 = np.hstack(super_final_oof_predictions2) *100
rsme = np.sqrt( np.mean( (oof2 - true)**2.0 ))
print('Overall CV SVR head RSME =',rsme)

oof_sk = np.hstack(super_final_oof_skSVR) *100
rsme = np.sqrt( np.mean( (oof_sk - true)**2.0 ))
print('Overall CV sklearn SVR head RSME =',rsme)

oof3 = (1-w)*oof + (w*oof2 + w*oof_sk)*0.5
rsme = np.sqrt( np.mean( (oof3 - true)**2.0 ))
print('Overall CV Ensemble heads RSME with 50% NN and 50% SVR =',rsme)

In [ ]:
import matplotlib.pyplot as plt

score = []
for ww in np.arange(0,1.05,0.05):
    oof3 = (1-ww)*oof + 0.5*(ww*oof2 + ww*oof_sk)
    rsme = np.sqrt( np.mean( (oof3 - true)**2.0 ))
    #print(f'{ww:0.2} CV Ensemble RSME =',rsme)
    score.append(rsme)
best_w = np.argmin(score)*0.05

plt.figure(figsize=(20,5))
plt.plot(np.arange(21)/20.0,score,'-o')
plt.plot([best_w],np.min(score),'o',color='black',markersize=15)
plt.title(f'Best Overall CV RSME={np.min(score):.4} with SVR Ensemble Weight={best_w:.2}',size=16)
plt.ylabel('Overall Ensemble RSME',size=14)
plt.xlabel('SVR Weight',size=14)
plt.show()

In [ ]:
df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
df_test['path'] = df_test['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')

df_test[['width', 'height', 'hw_ratio', 'wh_ratio', 'size']] = pd.DataFrame(scale.transform(df_test.apply(size_and_shape, axis=1).values))


In [ ]:
# ... oof

predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1) *100
predictions_sk = np.mean(np.column_stack(final_oof_predictions_sklearn), axis=1) *100

df_test["Pawpularity"] = (1-best_w)*predictions + 0.5 * (best_w*predictions2 + best_w*predictions_sk)
df_test = df_test[["Id", "Pawpularity"]]
df_test.to_csv("submission.csv", index=False)
df_test.head()
# operands could not be broadcast together with shapes (12432,) (8,) 

In [ ]:
predictions

In [ ]:
predictions2

In [ ]:
predictions_sk 

In [ ]:
df_test["Pawpularity"]